Packages

In [262]:
from pyomo.environ import *
import numpy as np
import random
import math
import pandas as pd
from numpy import flatnonzero as find
from pypower.api import case9, ext2int, bustypes, makeBdc, rundcpf, ppoption
from pypower.idx_bus import BUS_TYPE, REF, VA, PD, LAM_P, LAM_Q, MU_VMAX, MU_VMIN
from pypower.idx_gen import PG, MU_PMAX, MU_PMIN, MU_QMAX, MU_QMIN, PMAX, PMIN, GEN_BUS
from pypower.idx_brch import PF, PT, QF, QT, RATE_A, MU_SF, MU_ST
from pypower.idx_cost import COST
import matplotlib.pyplot as plt

In [263]:
# Run PyPower case to get initial condition
ppc = case9()
ppopt = ppoption(VERBOSE=0)
pp_dcpf, success = rundcpf(ppc, ppopt)
pp_bus = pp_dcpf["bus"]
pp_branch = pp_dcpf["branch"]
pp_gen = pp_dcpf["gen"]


Converged in 0.00 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0           0.0 to 0.0
Generators         3     On-line Capacity       820.0           0.0 to 0.0
Committed Gens     3     Generation (actual)    314.0               0.0
Loads              3     Load                   315.0               0.0
  Fixed            3       Fixed                315.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches           9     Losses (I^2 * Z)         0.00              0.00
Transformers       0     Branch Charging (inj)     -                0.0
Inter-ties         0     Total Inter-tie Flow     0.0               0.0
Areas     

In [264]:
## convert to internal indexing
ppc = ext2int(ppc)
baseMVA, bus, gen, branch, gencost = \
    ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"], ppc["gencost"]

## get bus index lists of each type of bus
ref, pv, pq = bustypes(bus, gen)
pvpq = np.matrix(np.r_[pv, pq])

## generator info
gbus = gen[:, GEN_BUS].astype(int)    ## what buses are they at?
refgen = find(gbus == ref)

## build B matrices and phase shift injections
B, Bf, _, _ = makeBdc(baseMVA, bus, branch)
B = B.todense()
Bf = Bf.todense()

# Problem dimensions
NG = gen.shape[0] # Number of generators
NB = bus.shape[0] # Number of buses
NBr = branch.shape[0] # Number of lines
NL = 3 # Number of loads
NW = 1 # Number of wind farms

# Map generator to bus
CG = np.zeros((NB,NG))
CG[gbus,range(3)] = 1

# Map load to bus
CL = np.zeros((NB,NL))
CL[4,0] = 1
CL[6,1] = 1
CL[8,2] = 1

# Map wind farm to bus
CW = np.zeros((NB,1))
CW[5,0] = 1

# Generator capacity limit p.u.
Pmax = gen[:, PMAX]/baseMVA
Pmin = gen[:, PMIN]/baseMVA

# Line flow limit p.u.
Lmax = branch[:, RATE_A]/baseMVA
Lmin = - branch[:, RATE_A]/baseMVA

# Quadratic cost coefficients
# Convert to p.u.
CG2 = gencost[:, COST]*baseMVA**2
CG1 = gencost[:, COST+1]*baseMVA
CG0 = gencost[:, COST+2]

# Cost coefficient for wind power
CW1 = 0
CW0 = 0

# Cost coefficients for generation reserve
CRG_up = np.zeros((NG,1))
CRG_down = np.zeros((NG,1))

# Cost coefficients for load reserve
CRL_up = 1000*np.ones((NL,1))
CRL_down = 1000*np.ones((NL,1))

# Uncontrollable load
PL_total = np.array([90,100,125])
PL = PL_total * 0.75

# Generator and demand set points
# Power in p.u., Va in rad
Pg0 = pp_gen[:, PG]/baseMVA
Pd0 = pp_bus[:, PD]/baseMVA
Pinj0 = (np.matmul(CG, Pg0) - Pd0)
Va0 = pp_bus[:, VA]/180*np.pi

In [265]:
# Read temperature data
weather_data = pd.read_csv("Weather_Data/NYC_Hourly_Normal_Temp.csv")
# Read wind power data
PW_data = pd.read_csv("spring_wind.csv",header=None).to_numpy()
PW_data = PW_data/500*75

In [266]:
Tamb = (weather_data.iloc[:,8] - 32)/1.8
Tamb_min = min(Tamb)
Tamb_10PCTL = (weather_data.iloc[:,6] - 32)/1.8
Tamb_90PCTL = (weather_data.iloc[:,7] - 32)/1.8
std_Tamb_10PCTL = (Tamb - Tamb_10PCTL)/1.282 
std_Tamb_90PCTL = (Tamb_90PCTL - Tamb)/1.282 
std_Tamb = (std_Tamb_10PCTL + std_Tamb_90PCTL)/2

In [267]:
def VS(dt,Tamb,num_TCL,Tset,delta,R,C,P_el,COP,baseMVA):
    Tdown = np.zeros(num_TCL)
    Tup = np.zeros(num_TCL)
    T_ON = np.zeros(num_TCL)
    T_OFF = np.zeros(num_TCL)
    D = np.zeros(num_TCL)
    B = 0
    Pdown = 0
    Pup = 0
    Sdown = 0
    Sup = 0
    for i in range(0,num_TCL):
        Tdown[i] = Tset[i] - delta/2
        Tup[i] = Tset[i] + delta/2
        T_ON[i] = C[i]*R[i]*np.log((Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])/(Tup[i] - Tamb - R[i]*P_el[i]*COP[i]))
        if ((Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])/(Tup[i] - Tamb - R[i]*P_el[i]*COP[i]) < 0):
            print(Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])
            print(Tup[i] - Tamb - R[i]*P_el[i]*COP[i])
        T_OFF[i] = C[i]*R[i]*np.log((Tup[i] - Tamb)/(Tdown[i] - Tamb))
        D[i] = T_ON[i]/(T_ON[i] + T_OFF[i])
        if ( Tamb < Tdown[i] and Tamb + R[i]*P_el[i]*COP[i] > Tup[i] ):
            B = B + P_el[i]*D[i]
            Pup = Pup + P_el[i]
            Sup = Sup + dt*P_el[i]*(1 - D[i])
    return [B/1000/baseMVA,Pdown/1000/baseMVA,Pup/1000/baseMVA,Sdown,Sup]

In [268]:
# Optimization: Problem formulation
def optimizer(NG,NL,Tamb_sample,PW_sample,PW_data,num_TCL,Tset,delta,R,C,P_el,COP,S0,PL,\
    Pmax,Pmin,Lmax,Lmin,CG,CW,CL,CG0,CG1,CG2,CW0,CW1,CRG_up,CRG_down,CRL_up,CRL_down):
    model = ConcreteModel()
    # Defining the variables
    model.PG = Var(range(NG),within = NonNegativeReals)
    model.PC = Var(range(NL),within = NonNegativeReals)
    model.RG_down = Var(range(NG),within = NonNegativeReals)
    model.RG_up = Var(range(NG),within = NonNegativeReals)
    model.RG = Var(range(NG),within = NonNegativeReals)
    model.RL_down = Var(range(NL),within = NonNegativeReals)
    model.RL_up = Var(range(NL),within = NonNegativeReals)
    model.RL = Var(range(NL),within = NonNegativeReals)
    model.dG_down = Var(range(NG),within = NonNegativeReals)
    model.dG_up = Var(range(NG),within = NonNegativeReals)
    model.dL_down = Var(range(NL),within = NonNegativeReals)
    model.dL_up = Var(range(NL),within = NonNegativeReals)
    # model.P_mis = Var(within=Reals)
    model.Va = Var(range(NB), within=Reals)

    model.dual = Suffix(direction=Suffix.IMPORT)

    # Defining the deterministic constraints
    model.c = ConstraintList()
    for i in range(NG):
        model.c.add(model.PG[i] + model.RG[i] >= Pmin[i])
        model.c.add(model.PG[i] + model.RG[i] <= Pmax[i])
        model.c.add(model.RG[i] >= -model.RG_down[i])
        model.c.add(model.RG[i] <= model.RG_up[i])
    model.c.add(sum(model.dG_down[i] for i in range(NG)) + sum(model.dL_up[j] for j in range(NL)) == 1)
    model.c.add(sum(model.dG_up[i] for i in range(NG)) + sum(model.dL_down[j] for j in range(NL)) == 1)
    
    # Line flow limit constraints
    for i in range(NBr):
        model.c.add(sum(Bf[i,j]*model.Va[j] for j in range(9)) <= Lmax[i])
        model.c.add(sum(Bf[i,j]*model.Va[j] for j in range(9)) >= Lmin[i])
    
    # Defining the CC constraints
    # for i in range(num_samples):
        # DC Power flow constraint
        # gen = np.zeros((NB,1))
        # wind = np.zeros((NB,1))
        # load = np.zeros((NB,1))
        # Pbus = np.zeros((NB,1))
        # for j in range(NB):
        #     gen[j] = sum(CG[j,k]*(model.PG[k] + model.RG[k]) for k in range(NG))
        #     wind[j] = CW[j]*PW_sample[i]
        #     load[j] = sum(CL[j,k]*(PL[k] + model.PC[k] + model.RL[k]) for k in range(NL))
        #     Pbus[j] = sum(B[j,k]*model.Va[k] for k in range(NB))
        #     model.c.add(gen[j] + wind[j] - load[j] == Pbus[j])
    i = 0
    P_mis_wind = 0
    PW_f = np.median(PW_data)
    P_mis_wind = P_mis_wind + (PW_sample[i] - PW_f)
    P_mis_load = 0
    for j in range(NL):
        identified_VS = VS(0.25,Tamb_sample[j],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
        P_mis_load = P_mis_load + (model.PC[j] - identified_VS[0])
    P_mis = P_mis_wind - P_mis_load
    # model.c.add(model.RG[i] == model.dG_up[i]*max(-model.P_mis,0) - model.dG_down[i]*max(model.P_mis,0))
    model.c.add(model.RG[i] == model.dG_up[i]*(-P_mis) - model.dG_down[i]*P_mis)
    for j in range(NL):
        # model.c.add(model.RL[j] == model.dL_up[j]*max(model.P_mis,0) - model.dL_down[j]*max(-model.P_mis,0))
        model.c.add(model.RL[j] == model.dL_up[j]*P_mis - model.dL_down[j]*(-P_mis))
        identified_VS = VS(0.25,Tamb_sample[j],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
        model.c.add(model.PC[i] + model.RL[i] >= identified_VS[1])
        model.c.add(model.PC[i] + model.RL[i] <= identified_VS[2])
        model.c.add(model.RL[i] >= -model.RL_down[i])
        model.c.add(model.RL[i] <= model.RL_up[i])
        model.c.add(S0[j] + 0.25*(model.PC[i] + model.RL[i] - identified_VS[0])*1000*baseMVA >= identified_VS[3])
        model.c.add(S0[j] + 0.25*(model.PC[i] + model.RL[i] - identified_VS[0])*1000*baseMVA  <= identified_VS[4])
        model.c.add(S0[j] + 0.75*(model.PC[i] - identified_VS[0])*1000*baseMVA  + 0.25*(model.PC[i] + model.RL[i] - identified_VS[0])*1000*baseMVA  >= identified_VS[3])
        model.c.add(S0[j] + 0.75*(model.PC[i] - identified_VS[0])*1000*baseMVA  + 0.25*(model.PC[i] + model.RL[i] - identified_VS[0])*1000*baseMVA  <= identified_VS[4])         
    
    def cost(model, CG0, CG1, CG2, CRG_down, CRG_up, CRL_down, CRL_up):
        cost = sum(CG0[i] for i in range(NG))\
            + sum(CG1[i]*model.PG[i] for i in range(NG))\
            + sum(CG2[i]*model.PG[i]**2 for i in range(NG))\
            + sum(CRG_up[i]*model.RG_up[i] for i in range(NG))\
            + sum(CRG_down[i]*model.RG_down[i] for i in range(NG))\
            + sum(CRL_up[i]*model.RL_up[i] for i in range(NL))\
            + sum(CRL_down[i]*model.RL_down[i] for i in range(NL))  
        return gencost

    model.obj = Objective(cost(model, CG0, CG1, CG2, CRG_down, CRG_up, CRL_down, CRL_up), sense=minimize)

    
    return model     

In [269]:
# Parameters related to the demand side
num_TCL = 3000
Tset = np.zeros((num_TCL,NL))
R = np.zeros((num_TCL,NL))
C = np.zeros((num_TCL,NL))
COP = np.zeros((num_TCL,NL))
P_el = np.zeros((num_TCL,NL))
Tset_set = [21,21.5,22,22.5,23]
for i in range(num_TCL): 
    for j in range(NL):
        s = random.sample(Tset_set,1)
        Tset[i,j] = s[0] 
delta = 1
R_nom = 2
R_low = 0.8*R_nom
R_high = 1.2*R_nom
C_nom = 2
C_low = 0.8*C_nom
C_high = 1.2*C_nom
COP_nom = 2.5
COP_low = 0.8*COP_nom
COP_high = 1.2*COP_nom
for i in range(NL): 
    R[:,i] = np.random.uniform(R_low,R_high,num_TCL)
    C[:,i] = np.random.uniform(C_low,C_high,num_TCL)
    COP[:,i] = np.random.uniform(COP_low,COP_high,num_TCL)
    for j in range(num_TCL):
        P_el[j,i] = np.ceil((Tset[j,i] + delta/2 - Tamb_min)/(R[j,i]*COP[j,i])) + 1
# Sampling
eps = 0.1
beta = 0.1
num_decision_variables = 5*NG + 5*NL
num_samples = np.ceil((2/eps)*(np.log(1/beta) + num_decision_variables)).astype(int)
i = 0
mu_Tamb = Tamb[i]
sigma_Tamb = std_Tamb[i] 
Tamb_sample = np.random.normal(mu_Tamb,sigma_Tamb,num_samples)
PW_sample = np.random.choice(PW_data[i,:], size=num_samples)

P_B = np.zeros((NL,1))
Pdown = np.zeros((NL,1))
Pup = np.zeros((NL,1))
Sdown = np.zeros((NL,1))
Sup = np.zeros((NL,1))
S0 = np.zeros((NL,1))

for j in range(NL):
    [P_B[j],Pdown[j],Pup[j],Sdown[j],Sup[j]] = \
        VS(0.25,Tamb[i],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
    S0[j] = (Sdown[j] + Sup[j])/2    


In [270]:

result = optimizer(NG,NL,Tamb_sample,PW_sample,PW_data,num_TCL,Tset,delta,R,C,P_el,COP,S0,PL,\
    Pmax,Pmin,Lmax,Lmin,CG,CW,CL,CG0,CG1,CG2,CW0,CW1,CRG_up,CRG_down,CRL_up,CRL_down)

TypeError: Unable to insert '[2.0e+00 1.5e+03 0.0e+00 3.0e+00 1.1e-01 5.0e+00 1.5e+02]' into Set FiniteScalarSet:
	TypeError: unhashable type: 'numpy.ndarray'